In [0]:
from google.colab import drive
import os
from IPython.display import clear_output
drive.mount('/content/gdrive', force_remount=True)
curDir =('/content/gdrive/My Drive/Colab Notebooks/DeepMovement_project')
os.chdir(curDir)
!cat /content/gdrive/My\ Drive/Colab\ Notebooks/DeepMovement_project

Mounted at /content/gdrive
cat: '/content/gdrive/My Drive/Colab Notebooks/DeepMovement_project': Is a directory


##Installation

In [0]:
!apt-get install swig

In [0]:
# !pip install openpose
os.chdir(curDir)
# ! git clone https://www.github.com/ildoonet/tf-pose-estimation
os.chdir('./tf-pose-estimation')
! pip install -r requirements.txt
os.chdir('./tf_pose/pafprocess')
! swig -python -c++ pafprocess.i && python setup.py build_ext --inplace
os.chdir(curDir)
clear_output

In [0]:
os.chdir(curDir)
!git clone https://www.github.com/ildoonet/tf-pose-estimation
os.chdir( './tf-pose-estimation')
! python setup.py install 
os.chdir(curDir)
clear_output
#restart

fatal: destination path 'tf-pose-estimation' already exists and is not an empty directory.
/content/gdrive/My: Operation not supported
running install
running bdist_egg
running egg_info
writing tf_pose.egg-info/PKG-INFO
writing dependency_links to tf_pose.egg-info/dependency_links.txt
writing requirements to tf_pose.egg-info/requires.txt
writing top-level names to tf_pose.egg-info/top_level.txt
file pafprocess.py (for module pafprocess) not found
writing manifest file 'tf_pose.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
file pafprocess.py (for module pafprocess) not found
copying tf_pose/pafprocess/pafprocess.py -> build/lib.linux-x86_64-3.6/tf_pose/pafprocess
file pafprocess.py (for module pafprocess) not found
running build_ext
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/tf_pose_data
copying build/lib.linux-x86_64-3.6/tf_pose_data/__init__.py -> build/bdist.linux-x86_64/egg/tf_pose_

##Process

In [0]:
import cv2
import os
import argparse
import logging
import sys
import time

#from tf_pose import common
#import numpy as np
import pandas as pd
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh
from UsedFunctions import manipulate, Get_Coords, Calculate_D, Calculate_L, Calculate_PCM, Calculate_TCM, Calculate_R, Add_Features_To_dataframe
%load_ext autoreload
%autoreload 2

W0627 22:45:35.053041 140565038032768 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_pose-0.1.1-py3.6-linux-x86_64.egg/tf_pose/mobilenet/mobilenet.py:369: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

Using TensorFlow backend.


In [0]:
def RUN(file_path, h_name, v_name=''):
    video_timing=time.time()             # For time computing
    print("Reading video file ", file_path)
    
    cap = cv2.VideoCapture(file_path)
    ret_val = True
    if cap.isOpened() is False:
        print("Error opening video stream or file")
    count = 0
    human_prev = None #@
    while ret_val:
        
        print ("Frame No: %d in %s" %(count, file_path))
        
        begin_counting=time.time()             # For time computing
        ret_val, image = cap.read()
        if not ret_val:
            break
        count += 1
        if count%2:
            print('skipping frame ')
            continue
            
        #image = cv2.imread("Output/Frame No_320.jpg")
        #start=time.time()
        #cv2.imwrite("Output/"+h_name+"_Frame_No_"+str(count)+".jpg",image)
        #print('Time taken for writing a Frame is {:.3f} ms'.format((time.time()-start)*1000))
        
        if image is None:
            logger.error('Image can not be read, path=%s' % file_path) #image)
            sys.exit(-1)
        
        humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=resize_out_ratio)
        #humans = e.inference(image)
        #print (humans)
        if len(humans)==0 or (mode=='train' and len(humans)>1):
            print("Inconvenient frame")
            continue
        Co_ordinates=manipulate(humans, human_prev, h_name, v_name, count-1, mode, predict_loc=True)     # Computes the Co-ordinates from the given data by O-Nect
        human_prev = Co_ordinates.iloc[0].copy()#@
        
        #print('Time taken for Co-ordinate calculations for {} Frames is {:.3f} ms'.format(len(Co_ordinates),(time.time()-begin_counting)*1000))
        
        # Extracting X, Y Coordinates
        #start=time.time()
        X_Coords, Y_Coords= Get_Coords(Co_ordinates)
        #print('Time taken for Co-ordinate extractions for {} Frames is {:.3f} ms'.format(len(Co_ordinates),(time.time()-start)*1000))
        
        #start=time.time()
        PCM_Frames= Calculate_PCM(X_Coords, Y_Coords)
        #print('Time taken for PCM for {} Frames is {:.3f} ms'.format(len(PCM_Frames),(time.time()-start)*1000))
        
        #start=time.time()
        TCM_x, TCM_y= Calculate_TCM(PCM_Frames)
        #print('Time taken for TCM for {} Frames is {:.3f} ms'.format(len(TCM_x),(time.time()-start)*1000))
        
        #start=time.time()
        L= Calculate_L(TCM_x, TCM_y, PCM_Frames)
        #print('Time taken for L features for {} Frames is {:.3f} ms'.format(len(PCM_Frames),(time.time()-start)*1000))
        
        #start=time.time()
        D1, D2, D3 = Calculate_D(PCM_Frames, TCM_x, TCM_y, 'Degrees')
        #print('Time taken for D1, D2, D3 features for {} Frames is {:.3f} ms'.format(len(PCM_Frames),(time.time()-start)*1000))
        
        #start=time.time()
        R=Calculate_R(PCM_Frames)
        #print('Time taken for R feature for {} Frames is {:.3f} ms'.format(len(PCM_Frames),(time.time()-start)*1000))
        
        #start=time.time()
        out=Add_Features_To_dataframe(Co_ordinates, PCM_Frames, TCM_x, TCM_y, L, R, D1, D2, D3)
        #print('Time taken for adding features to dataframe for {} Frames is {:.3f} ms'.format(len(Co_ordinates),(time.time()-start)*1000))
        
        print('Time taken for the whole file of {} Frames is {:.3f} ms'.format(len(PCM_Frames),(time.time()-begin_counting)*1000))
        #count = count + 1
        OUTPUTS.append(out)
        if not count%100:
            r = pd.concat(OUTPUTS)
            r.to_csv(OUTPUT_PATH, index=False)
    #print('Time taken for the whole {} file is {:.3f} minuts'.format(v if 'v' in locals() else f, (time.time()-video_timing)/60))
    print('Time taken for the whole {} file is {:.3f} minuts'.format(file_path, (time.time()-video_timing)/60))
    return OUTPUTS

In [0]:
""" write your own path here """
videos_path = 'Actions_set'


logger = logging.getLogger('TfPoseEstimator')
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s] [%(name)s] [%(levelname)s] %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)


# parser = argparse.ArgumentParser(description='tf-pose-estimation Video')
# parser.add_argument('--video-directory', type=str, default=videos_path,
#                     help='if provided, select videos in that directory. ')
# parser.add_argument('--resolution', type=str, default='432x368', help='network input resolution. default=432x368')
# parser.add_argument('--model', type=str, default='mobilenet_thin', help='cmu / mobilenet_thin')
# parser.add_argument('--show-process', type=bool, default=False,
#                     help='for debug purpose, if enabled, speed for inference is dropped.')
# parser.add_argument('--resize', type=str, default='0x0',
#                     help='if provided, resize images before they are processed. default=0x0, Recommends : 432x368 or 656x368 or 1312x736 ')
# parser.add_argument('--showBG', type=bool, default=True, help='False to show skeleton only.')
# parser.add_argument('--resize-out-ratio', type=float, default=4.0,
#                     help='if provided, resize heatmaps before they are post-processed. default=1.0')
# parser.add_argument('--mode', type=str, default='test',
#                     help='if provided, choose the run mode to be eather train or test. default=test ')
# args = parser.parse_args()

video_directory = videos_path
resolution = '432x368'
model = 'mobilenet_thin'
show_process = False
resize = '0x0'
showBG = True
resize_out_ratio = 4.0
mode = 'train'



In [0]:
total_timing=time.time()             # For time computing

# logger.debug('initialization %s : %s' % (model, get_graph_path(model)))

w, h = model_wh(resize)
if w == 0 or h == 0:
	e = TfPoseEstimator(get_graph_path(model), target_size=(432, 368))
else:
	e = TfPoseEstimator(get_graph_path(model), target_size=(w, h))
        
newpath=os.path.join(os.getcwd(),'Output')
if not os.path.exists(newpath):
    os.makedirs(newpath)
OUTPUT_PATH=os.path.join(newpath,'Activity_'+mode+'with_NaNs.csv')

OUTPUTS = []

# df = pd.read_csv(OUTPUT_PATH)
# df = df[df.V_title != 'person22_walking_d3_uncomp']
# OUTPUTS.append(df)
# flag = 0

print("Initiating {} mode...".format(mode))

# optional: to activate un comment the firs section in the for loop
valid_videos = [".avi",".mp4",".mov",".mkv",".rmvb", ".dv", ".ts"] 
for f in os.listdir(video_directory):
    print(f)
#     if f != 'walking':
#         continue
    h_name=os.path.splitext(f)[0]
    if os.path.splitext(f)[1] == '': # is it a directory?
#         if f != 'P15' and flag == 0:
#             continue
#         else:
#             flag=1
        for v in os.listdir(os.path.join(video_directory,f)):
            #"""
            # for including only video formates
            #if v != 'person22_walking_d3_uncomp.avi' and flag == 0:
            #    continue
            #else:
            #    flag=1
            ext = os.path.splitext(v)[1]
            if ext.lower() not in valid_videos:
                continue
            #"""
            OUTPUTS = RUN(os.path.join(video_directory,f, v), str(h_name), str(os.path.splitext(v)[0]))
        if 'v' in globals(): del v 
    else:
        #"""
        # for including only video formates
        ext = os.path.splitext(f)[1]
        if ext.lower() not in valid_videos:
            continue
        #"""
        OUTPUTS = RUN(os.path.join(video_directory,f), str(h_name))
result = pd.concat(OUTPUTS)
result.to_csv(OUTPUT_PATH, index=False)
print('Total time taken is {:.3f} minuts'.format((time.time()-total_timing)/60))


Output hidden; open in https://colab.research.google.com to view.

In [0]:
x = list(range(20))
xx = x + 

IndexError: ignored